## Physics arxiv's DOI retreived - 603.178 DOIs

In [2]:
path = "------------------------------" #Path has removed for confidentiality reasons
physics_doi= sqlContext.read.format('csv').options(header='true', inferSchema='true').option("delimiter", ",").load(path)
physics_doi.write.mode("overwrite").saveAsTable("taxiarchis.physics_doi_csv")
display(physics_doi)

## Combine DOIs with the scopus papers

In [4]:
%scala
//DONT EXECUTE , ONLY IF YOU WANT TO TAKE A SUBSET
// val number_of_articles = 10000  //Select the number of articles to be retreived out of 64.500 DOIs

val arxiv = table("taxiarchis.physics_doi_csv").withColumnRenamed("DOI", "DOI_arxiv")


val sd_old = table("-----------") // #Mapping table name has removed for confidentiality reasons
val joined = arxiv.join(sd_old, sd_old("doi") === arxiv("DOI_arxiv"), "inner")

val xmls   = "----------------------------" #Path has removed for confidentiality reasons
val sd = sc.sequenceFile[String, String](xmls)

case class SDmeta  (
                     pii_unformatted: String, 
                     content: String
)

import org.apache.spark.sql.functions._
import scala.collection.JavaConverters._

// Map the transformed records to the CAR record  and create a CAR dataset
var metaDS = sd.map(rec => new SDmeta(  rec._1, rec._2 )).toDF

val final_joined = joined.join(metaDS, joined("pii_unformatted") === metaDS("pii_unformatted"), "inner").select("DOI_arxiv", "content", "Categories")


def generateHTMLLinks(pathInsideFileStore: String): String = {
  /* path should be inside FileStore, but can remove the dbfs:/FileStore from the path if needed */
  val pathInsideFileStore_clean=pathInsideFileStore.replace("-------------","")#Path has removed for confidentiality reasons
  dbutils
  .fs
  .ls(s"----------------------------------") #Path has removed for confidentiality reasons
  .map(_.name)
  .map(
    item => s"""-------------------------------------""" #Path has removed for confidentiality reasons
  ).mkString(
    ""
  )
}

display(final_joined)

## Select the categories you want

In [6]:
%scala
val test =final_joined.filter((final_joined("Categories") === "Condensed Matter - Statistical Mechanics") || (final_joined("Categories")  === "Condensed Matter - Mesoscale and Nanoscale Physics") || (final_joined("Categories") === "Condensed Matter - Materials Science" ) || (final_joined("Categories") === "Condensed Matter - Strongly Correlated Electrons" ))
test.count()

##Download as .csv

In [8]:
%scala

test.repartition(1).write.mode("overwrite").format("com.databricks.spark.csv").option("header", "true").option("delimiter", "\t").option("charset", "utf-8").option("quote", "\u0000").save("dbfs:/FileStore/Taxiarchis-data-four_categories")


displayHTML(generateHTMLLinks("dbfs:/FileStore/Taxiarchis-data-four_categories"))

## These are the main tables created for the final experiments - 4 categories

###DOI and Categories

In [11]:
path = "---------------------------" #Path has removed for  confidentiality reasons
df_spark_doi_cat= sqlContext.read.format('csv').options(header='true', inferSchema='true').option("delimiter", "\t").load(path).cache()
display(df_spark_doi_cat)

### 4 Categories - Load Datasets

In [13]:

#DOI_Categories_4 ------> Contains the mappingdois with the write category
#FullDocument_4 ------> Contains the documents as a full text  
#perSection_4_1------> Contains all the sections/subsections  of all documents 

path1 = "---------------------------" #Path has removed for confidentiality reasons
path2 = "---------------------------" #Path has removed for confidentiality reasons
path3 = "---------------------------" #Path has removed for confidentiality reasons



#DOI_Categories_4
df_DOI_Categories_4= sqlContext.read.format('csv').options(header='true', inferSchema='true').option("delimiter", "\t").load(path1)

## Write Frame out as Table
df_DOI_Categories_4.write.mode("overwrite").saveAsTable("taxiarchis.DOI_Categories_4")


# FullDocument_4
df_FullDocument_4= sqlContext.read.format('csv').options(header='true', inferSchema='true').option("delimiter", "\t").load(path2)

## Write Frame out as Table
df_FullDocument_4.write.mode("overwrite").saveAsTable("taxiarchis.FullDocument_4")



#perSection_4_1
df_perSection_4_1 = sqlContext.read.format('csv').options(header='true', inferSchema='true').option("delimiter", "\t").load(path3)

## Write Frame out as Table
df_perSection_4_1.write.mode("overwrite").saveAsTable("taxiarchis.perSection_4_1")


### Dataset of sections for the ensemble tested method

In [15]:
#Section_4_4_Ensemble------> Contains all the sections/subsections  of all documents. Each document have been split in four parts 1) Abstract 2) Intrroduction 3) in detween 4)Conclusion 

path_ensembele =  "---------------------------" #Path has removed for confidentiality reasons

#perSection_4_1
df_perSection_4_4 = sqlContext.read.format('csv').options(header='true', inferSchema='true').option("delimiter", "\t").load(path_ensembele)

## Write Frame out as Table
df_perSection_4_4.write.mode("overwrite").saveAsTable("taxiarchis.Section_4_4_Ensemble")

### The mapping of the DOIs with the categories

In [17]:
path_doi =  "---------------------------" #Path has removed  for confidentiality reasons

#perSection_4_1
df_perSection_4_4_1 = sqlContext.read.format('csv').options(header='true', inferSchema='true').option("delimiter", "\t").load(path_doi)

## Write Frame out as Table
df_perSection_4_4_1.write.mode("overwrite").saveAsTable("taxiarchis.doi_categories_4_4_features")